In [167]:
import pandas as pd
import math
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm

# First solution

In [173]:
# Function to find the closest unvisited city using geodesic distance
def find_closest_city(data, start_coord, visited_cities):
    # Initialize variables to store the closest city
    min_distance = float('inf')
    closest_city = None
    closest_city_coord = None
    
    # Loop through each city to calculate the geodesic distance
    for index, row in data.iterrows():
        city_name = row[0]
        city_coord = (row[1], row[2])  
        
        # Skip the city if it's already visited
        if city_name in visited_cities:
            continue
        
        # Calculate geodesic distance between start_coord and city_coord
        distance = geodesic(start_coord, city_coord).kilometers
        
        # Update the closest city if this one is closer
        if distance < min_distance and distance != 0:
            min_distance = distance
            closest_city = city_name
            closest_city_coord = city_coord
    
    return closest_city, closest_city_coord, min_distance
    

# Function to solve the TSP using geodesic distance and nearest neighbor
def tsp_nearest_neighbor(data, start_city, start_city_coord):
    
    # Initialize visited cities array
    visited_cities = []
    
    # Add starting city to the visited list
    visited_cities.append(start_city)
    
    #print(f"Starting city: {start_city}")
    
    total_distance = 0
    current_city_coord = start_city_coord
    
    # Loop until all cities are visited
    with tqdm(total=len(data) - 1, desc="Visiting cities") as pbar:
        while len(visited_cities) < len(data):
            # Find the closest unvisited city
            closest_city, closest_city_coord, min_distance = find_closest_city(data, current_city_coord, visited_cities)
            
            # Add the closest city to the visited cities array
            visited_cities.append(closest_city)
            
            # Update the current city coordinates to the new closest city's coordinates
            current_city_coord = closest_city_coord
            
            # Accumulate the total distance
            total_distance += min_distance
            
            #print(f"Travels to {closest_city} with a distance of {min_distance:.2f} km")
            
            # Update the progress bar
            pbar.update(1)
    
    # Return to the start city to complete the loop
    return_to_start_distance = geodesic(current_city_coord, start_city_coord).kilometers
    total_distance += return_to_start_distance
    visited_cities.append(start_city)  # Add the starting city at the end to close the loop
    
    #print(f"Returning to {start_city} with a distance of {return_to_start_distance:.2f} km")

    
    return visited_cities, total_distance

# Second solution

In [179]:
# Calculate vectors (distance and bearing) from the start city to all other cities
def calculate_vectors(data, start_city_name, start_city_coord):
    vectors = []
    for index, row in data.iterrows():
        city_name = row[0]
        city_coord = (row[1], row[2])
        
        if city_name == start_city_name:
            continue
        
        distance = geodesic(start_city_coord, city_coord).kilometers
        bearing = calculate_bearing(start_city_coord, city_coord)
        
        vectors.append({
            'city': city_name,
            'coordinates': city_coord,
            'distance': distance,
            'bearing': bearing
        })
    
    # Sort by bearing
    vectors = sorted(vectors, key=lambda x: x['bearing'])
    return vectors

# Helper function to calculate bearing (angle in relation to North)
def calculate_bearing(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon))
    initial_bearing = math.atan2(x, y)
    
    bearing = (math.degrees(initial_bearing) + 360) % 360
    return bearing

# Find the largest bearing difference
def find_largest_bearing_difference(vectors):
    max_difference = 0
    city_pair = None
    
    for i in range(1, len(vectors)):
        current_bearing = vectors[i]['bearing']
        previous_bearing = vectors[i - 1]['bearing']
        
        difference = current_bearing - previous_bearing
        
        if difference > max_difference:
            max_difference = difference
            city_pair = (vectors[i - 1], vectors[i])
    
    circular_difference = 360 - vectors[-1]['bearing'] + vectors[0]['bearing']
    if circular_difference > max_difference:
        max_difference = circular_difference
        city_pair = (vectors[-1], vectors[0])
    
    return city_pair

# Calculate total distance of the route and print city order
def calculate_total_distance_and_route(start_city_name, start_coord, vectors, city_pair):
    total_distance = 0.0
    route = [start_city_name]

    #print(f"Starting city: {start_city_name}")
    
    # Start from the second city in the largest bearing difference pair
    route.append(city_pair[1]['city'])
    total_distance += geodesic(start_coord, city_pair[1]['coordinates']).kilometers

    #print(f"Travels to {city_pair[1]['city']} with a distance of {total_distance:.2f} km")
    
    # Next city, closest in terms of bearing
    next_index = vectors.index(city_pair[1]) + 1
    
    # Cover all cities and stopping at the first city in the largest bearing difference pair
    while route[-1] != city_pair[0]['city']:

        # Go to the begining of the list of sorted vectors when the end is reached
        if next_index > (len(vectors)-1):
            next_index = 0
        
        route.append(vectors[next_index]['city'])
        distance = geodesic(vectors[next_index - 1]['coordinates'], vectors[next_index]['coordinates']).kilometers
        total_distance += distance

        #print(f"Travels to {vectors[next_index]['city']} with a distance of {distance:.2f} km")

        next_index += 1


    # Add distance from the first city in the pair back to the starting city
    total_distance += geodesic(start_coord, city_pair[0]['coordinates']).kilometers
    route.append(start_city_name)
    
    return total_distance, route

# Main function
def solve_tsp_with_vectors(data, start_city, start_city_coord):
    
    # Calculate vectors
    vectors = calculate_vectors(data, start_city, start_city_coord)
    
    # Find the largest bearing difference
    city_pair = find_largest_bearing_difference(vectors)
    
    # Calculate total distance traveled and route
    total_distance, route = calculate_total_distance_and_route(start_city, start_city_coord, vectors, city_pair)
    print('------------------------------------------------------------')
    print(f"Total distance traveled: {total_distance:.2f} km")
    print("Travel order:", " -> ".join(route))
    
    return total_distance, route


# Results

In [175]:
# Import CSV file
def import_cities(filename):
    return pd.read_csv(filename)

# Choose a random starting city
def choose_random_city(data):
    random_index = np.random.choice(data.index)
    city_name = data.iloc[random_index][0]
    city_coord = (data.iloc[random_index][1], data.iloc[random_index][2])  
    return city_name, city_coord

In [181]:
data = import_cities('vanuatu.csv')
#data = import_cities('italy.csv')
#data = import_cities('us.csv')
#data = import_cities('russia.csv')
#data = import_cities('china.csv')
start_city, start_city_coord = choose_random_city(data)

In [182]:
# First solution
visited_cities, total_distance = tsp_nearest_neighbor(data, start_city, start_city_coord)
print('------------------------------------------------------------')
print(f"Total distance traveled: {total_distance:.2f} km")
travel_order = " -> ".join(visited_cities)
print(f"Travel order: {travel_order}")

Visiting cities: 100%|██████████| 6/6 [00:00<00:00, 397.66it/s]

------------------------------------------------------------
Total distance traveled: 1158.19 km
Travel order: Port Olry -> Luganville -> Norsup -> Lakatoro -> Longana -> Sola -> Vila -> Port Olry


In [183]:
# Second solution
total_distance, route = solve_tsp_with_vectors(data, start_city, start_city_coord)

------------------------------------------------------------
Total distance traveled: 907.09 km
Travel order: Port Olry -> Sola -> Longana -> Vila -> Lakatoro -> Norsup -> Luganville -> Port Olry
